In [1]:
# this lets us use the figures interactively
%matplotlib inline

import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

plt.ion() # make the plotting interactive

In [2]:
# generate matplotlib handles to create a legend of the features we put in our map.
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles

In [3]:
def scale_bar(ax, location=(0.32, 0.95)):
    x0, x1, y0, y1 = ax.get_extent()
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=9, transform=ax.projection)
    ax.plot([sbx, sbx - 5000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    ax.plot([sbx-5000, sbx - 10000], [sby, sby], color='w', linewidth=6, transform=ax.projection)

    ax.text(sbx, sby-1000, '10 km', transform=ax.projection, fontsize=10)
    ax.text(sbx-5000, sby-1000, '5 km', transform=ax.projection, fontsize=10)
    ax.text(sbx-8000, sby-1000, '0 km', transform=ax.projection, fontsize=10)


In [4]:
#import data file: flood, buildings, roads, population excel file, county outline, sa outline
flood= gpd.read_file(os.path.abspath('Project_datafiles/Flood_2m.shp'))
roads= gpd.read_file(os.path.abspath('Project_datafiles/Fermanagh_roads.shp'))
buildings= gpd.read_file(os.path.abspath('Project_datafiles/Building_Fermanagh.shp'))
pop_demography=gpd.read_file(os.path.abspath('Project_datafiles/pop_demography.csv'))
outline = gpd.read_file(os.path.abspath('Project_datafiles/Fermanagh_DCA.shp'))
small_area= gpd.read_file(os.path.abspath('Project_datafiles/SA_poly.shp'))
land_cover= gpd.read_file(os.path.abspath('Project_datafiles/LC_Fermanagh.shp'))

In [5]:
small_area =gpd.GeoDataFrame(pop_demography.merge(small_area, on="SA_Code"))
small_area.head()

,SA_Code,Age_75,Children_9,residents,geometry_x,SA,Shape_Leng,Shape_Area,geometry_y
0,N00002912,25,38,345,None,Ballinamallard ward,26671.860028,1.255163e+07,"POLYGON ((227161.047 350213.156, 227161.109 35..."
1,N00002921,21,31,309,None,Belcoo and Garrison ward,62798.682569,8.317093e+07,"POLYGON ((206591.234 353513.594, 206598.359 35..."
2,N00002922,33,65,468,None,Belcoo and Garrison ward,47129.291829,2.964456e+07,"POLYGON ((195507.891 357313.719, 195458.531 35..."
3,N00002924,19,31,386,None,Belleek and Boa ward,6016.303226,6.970221e+05,"POLYGON ((194939.719 358792.812, 194937.359 35..."
4,N00002925,20,43,311,None,Belleek and Boa ward,19160.475195,7.193503e+06,"POLYGON ((197286.328 359952.875, 197299.969 35..."


In [6]:
small_area= small_area.set_geometry('geometry_y')
small_area.geometry

0     POLYGON ((227161.047 350213.156, 227161.109 35...
1     POLYGON ((206591.234 353513.594, 206598.359 35...
2     POLYGON ((195507.891 357313.719, 195458.531 35...
3     POLYGON ((194939.719 358792.812, 194937.359 35...
4     POLYGON ((197286.328 359952.875, 197299.969 35...
5     POLYGON ((217257.859 365303.062, 217235.938 36...
6     POLYGON ((212900.172 360800.438, 212843.938 36...
7     POLYGON ((220325.609 345147.219, 220319.203 34...
8     POLYGON ((224635.609 343878.469, 224633.797 34...
9     POLYGON ((226659.703 344481.781, 226657.734 34...
10    POLYGON ((219265.688 352138.938, 219333.734 35...
11    POLYGON ((214762.625 357207.594, 214790.938 35...
12    POLYGON ((223555.719 345060.031, 223554.516 34...
13    POLYGON ((223486.719 344807.031, 223465.156 34...
14    POLYGON ((223129.719 345194.031, 223142.109 34...
15    POLYGON ((224110.750 344338.406, 224110.234 34...
16    POLYGON ((223045.547 347631.688, 223045.297 34...
17    POLYGON ((223980.281 345782.031, 223981.00

In [7]:
small_area['residents'] = small_area['residents'].astype(int)
small_area['Shape_Area'] = small_area['Shape_Area'].astype(int)


In [10]:
for ind, row in small_area.iterrows(): # iterate over each row in the GeoDataFrame
    small_area.loc[ind, 'population_density'] = row['residents']/ row['Shape_Area'] * 1000000
print(small_area.head())

     SA_Code Age_75 Children_9  residents geometry_x   
0  N00002912     25         38        345       None  \
1  N00002921     21         31        309       None   
2  N00002922     33         65        468       None   
3  N00002924     19         31        386       None   
4  N00002925     20         43        311       None   

                         SA    Shape_Leng  Shape_Area   
0       Ballinamallard ward  26671.860028    12551625  \
1  Belcoo and Garrison ward  62798.682569    83170934   
2  Belcoo and Garrison ward  47129.291829    29644558   
3      Belleek and Boa ward   6016.303226      697022   
4      Belleek and Boa ward  19160.475195     7193502   

                                          geometry_y  population_density  
0  POLYGON ((227161.047 350213.156, 227161.109 35...           27.486481  
1  POLYGON ((206591.234 353513.594, 206598.359 35...            3.715240  
2  POLYGON ((195507.891 357313.719, 195458.531 35...           15.787046  
3  POLYGON ((194939.

In [ ]:
flood.to_crs(epsg = 2157)
roads.to_crs(epsg = 2157)
buildings.to_crs(epsg = 2157)
outline.to_crs(epsg = 2157)
small_area.to_crs(epsg = 2157)
land_cover.to_crs(epsg = 2157)

In [ ]:
def underwater(shapefile):
    flood_geom = flood['geometry'].values[0]
    underwater = shapefile['geometry'].within(flood_geom)
    return shapefile['geometry'].within(flood_geom)

In [ ]:
lc_flooded = underwater(land_cover) # call underwater function to select land parcels in 5m flood zone
land_flooded = land_cover[lc_flooded] # subset the land cover shapefile to include only underwater = true
print(land_flooded)

In [ ]:
myFig = plt.figure(figsize=(12, 12))  # create a figure of size 10x10 (representing the page size in inches)

myCRS = ccrs.UTM(29)  # create a Universal Transverse Mercator reference system to transform our data.
# be sure to fill in XX above with the correct number for the UTM Zone that Northern Ireland is part of.

ax = plt.axes(projection=myCRS)  # finally, create an axes object in the figure, using a UTM projection,
# where we can actually plot our data.

In [ ]:
# first, we just add the outline of Northern Ireland using cartopy's ShapelyFeature
outline_feature = ShapelyFeature(outline['geometry'], myCRS, edgecolor='k', facecolor='w')
xmin, ymin, xmax, ymax = flood.total_bounds
ax.add_feature(outline_feature) # add the features we've created to the map.
ax.set_extent([xmin-1000, xmax+1000, ymin-1000, ymax+1000], crs=myCRS)


In [ ]:
# get the number of unique municipalities we have in the dataset
landcover_type = len(land_cover.LAND_COVER.unique())
print('Number of unique features: {}'.format(landcover_type))
landcover=list(land_cover.LAND_COVER.unique())

In [ ]:
landcover_flooded = len(land_flooded.LAND_COVER.unique())
print('Number of unique features: {}'.format(landcover_flooded))
landflooded=list(land_flooded.LAND_COVER.unique())

In [ ]:
landcover_colors = ['springgreen','olive', 'sienna','darkred', 'lawngreen', 'forestgreen','yellowgreen', 'y','darkgreen','darkorange','lightgrey','gold','black','grey']
for ii, name in enumerate(landcover):
    feat = ShapelyFeature(land_cover.loc[land_cover['LAND_COVER'] == name, 'geometry'], # first argument is the geometry
                          myCRS, # second argument is the CRS
                          edgecolor='k', # outline the feature in black
                          facecolor=landcover_colors[ii],
                        linewidth=0.1,
                         alpha=0.8) # set the face color to the corresponding color from the list
                           # set the outline width to be 1 pt
                           # set the alpha (transparency) to be 0.25 (out of 1)
    ax.add_feature(feat)


In [ ]:
flood_feature = ShapelyFeature(flood['geometry'], myCRS, edgecolor='k',facecolor='navy',alpha=0.1,linewidth=0.1)
ax.add_feature(flood_feature)


In [ ]:
land_flooded_colors = ['grey','darkgreen','forestgreen','lightgrey', 'gold','sienna','springgreen','lawngreen','olive','black']
for ii, name in enumerate(landflooded):
    feat = ShapelyFeature(land_flooded.loc[land_flooded['LAND_COVER'] == name, 'geometry'], # first argument is the geometry
                          myCRS, # second argument is the CRS
                          edgecolor='r', # outline the feature in black
                          facecolor=land_flooded_colors[ii],
                        linewidth=0.2,
                         alpha=1)
                          # set the face color to the corresponding color from the list
                           # set the outline width to be 1 pt
                           # set the alpha (transparency) to be 0.25 (out of 1)
    ax.add_feature(feat)


In [ ]:
landcover_handles = generate_handles(land_cover.LAND_COVER.unique(), landcover_colors)
flood_handles = generate_handles(['Flood'], ['navy'])

In [ ]:
# ax.legend() takes a list of handles and a list of labels corresponding to the objects
# you want to add to the legend
handles = landcover_handles  + flood_handles # use '+' to concatenate (combine) lists
labels = landcover + ['Flood']

leg = ax.legend(handles, labels, title='Legend', title_fontsize=12,
                 fontsize=10, loc='lower left', frameon=True, framealpha=1)

gridlines = ax.gridlines(draw_labels=True, alpha=1, edgecolor='k') # draw  labels for the grid lines
                         
gridlines.left_labels = False # turn off the left-side labels
gridlines.bottom_labels = False

scale_bar(ax)
myFig # to show the updated figure

In [ ]:
myFig2 = plt.figure(figsize=(12, 12))  # create a figure of size 10x10 (representing the page size in inches)

myCRS = ccrs.UTM(29)  # create a Universal Transverse Mercator reference system to transform our data.
# be sure to fill in XX above with the correct number for the UTM Zone that Northern Ireland is part of.

ax = plt.axes(projection=myCRS)  # finally, create an axes object in the figure, using a UTM projection,
# where we can actually plot our data.

In [ ]:
#create % column for elderly and child ?

In [ ]:
#create column pop density

In [ ]:
#all data shapefile clipped to flood polygon with within

In [ ]:
#analyses:-tot pop in flood polygon
#         - nbr resident houses
#         - tot length of road impacted and class A roas
#         - Sa with highest pop density and if any are adjacent
#         - SA with less than avg pop density
#         - which landcover types has highest area in flood poly
#         - total area flooded per landcover types
#         - SA with highest % of elderly/child
#         - SA with low pop density but high child %
#         - SA with medical building
#         - residential building furtherest from medical build (use original dataset ?)
#         - SA centre furthest from medical building

In [ ]:
#add scale and legend handle

In [ ]:
#create map and add features to map
# do second map with landcover

In [ ]:
#different labels depending on building class and road class

In [ ]:
#grid lines